# Week 3: LLM Integration and Prompts

**Pipeline continuity:**
- **Week 1 output:** model/metric decisions (`MPNet` operational, `cosine` champion).
- **Week 2 output:** working local retriever (PDF -> chunk -> embed -> FAISS -> top-k).
- **Week 3 focus:** keep the same retrieval layer and add generation (prompting + local LLM).

**Goal:** end-to-end RAG flow for one query cycle:
`query -> retrieve context (Week 2) -> build prompt -> LLM answer`.


In [1]:
import json
import time
import hashlib
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate

faiss.omp_set_num_threads(4)

/Users/tkhamidulin/Desktop/First Project - RAG/rag_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Same paths as Week 2 (notebook is in notebooks/)
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

W3_EXPERIMENTS_CSV = ARTIFACTS_DIR / "week3_prompt_experiments.csv"
W3_GENERATIONS_JSONL = ARTIFACTS_DIR / "week3_generations.jsonl"

print(f"Project root: {PROJECT_ROOT}")
print(f"Data dir: {DATA_DIR} (exists: {DATA_DIR.exists()})")
print(f"Artifacts: {ARTIFACTS_DIR}")


(PosixPath('/Users/tkhamidulin/Desktop/First Project - RAG/artifacts/week2_retrieval_debug.jsonl'),
 PosixPath('/Users/tkhamidulin/Desktop/First Project - RAG/artifacts'))

---
## From Week 1 and Week 2: Retrieval Pipeline

**From Week 1:** Используем ту же модель эмбеддингов (MPNet) и метрику (cosine via normalized embeddings + IndexFlatIP).

**From Week 2:** Тот же путь данных, загрузка PDF, чанкинг и классы VectorStore и RAG. Мы повторяем их здесь, чтобы Week 3 работал сам по себе и явно опирался на Week 2.


In [ ]:
# --- From Week 2: load PDFs and chunking ---
def load_pdfs(data_dir: Path):
    docs = []
    for topic_dir in sorted(data_dir.iterdir()):
        if not topic_dir.is_dir():
            continue
        topic = topic_dir.name
        for pdf_path in topic_dir.glob("*.pdf"):
            try:
                loader = PyPDFLoader(str(pdf_path))
                for doc in loader.load():
                    doc.metadata["topic"] = topic
                    doc.metadata["source"] = pdf_path.name
                    docs.append(doc)
            except Exception as e:
                print(f"  ERROR: {pdf_path.name} - {e}")
    return docs

def chunk_documents(docs, chunk_size: int, chunk_overlap: int, separators=None):
    if separators is None:
        separators = ["\n\n", "\n", ". ", " ", ""]
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, separators=separators)
    return splitter.split_documents(docs)

# --- From Week 2: VectorStore and RAG (Week 1 model + cosine) ---
EMBEDDING_MODEL_ID = "all-mpnet-base-v2"

class VectorStore:
    def __init__(self, model_id: str = EMBEDDING_MODEL_ID):
        self.model = SentenceTransformer(model_id)
        self.chunks = []
        self._index = None

    def build_index(self, chunks):
        self.chunks = list(chunks)
        if not self.chunks:
            raise ValueError("Cannot index: chunks list is empty")
        texts = [c.page_content for c in self.chunks]
        emb = self.model.encode(texts, normalize_embeddings=True, show_progress_bar=True)
        emb = np.ascontiguousarray(emb, dtype=np.float32)
        self._index = faiss.IndexFlatIP(emb.shape[1])
        self._index.add(emb)

    def retrieve(self, query: str, top_k: int = 3):
        if self._index is None or self._index.ntotal == 0:
            raise ValueError("Index is empty; call build_index(chunks) first")
        q = self.model.encode([query], normalize_embeddings=True)
        q = np.ascontiguousarray(q, dtype=np.float32)
        k = min(top_k, self._index.ntotal)
        scores, indices = self._index.search(q, k)
        return scores[0], indices[0]

class RAG:
    def __init__(self, vector_store: VectorStore):
        self.vector_store = vector_store

    def retrieve(self, query: str, top_k: int = 3):
        scores, indices = self.vector_store.retrieve(query, top_k=top_k)
        return [(float(s), self.vector_store.chunks[i]) for s, i in zip(scores, indices)]

print("Loaded: load_pdfs, chunk_documents, VectorStore, RAG (same as Week 2). Embedding model:", EMBEDDING_MODEL_ID)


In [ ]:
# Run Week 2 pipeline: load PDFs -> chunk -> build index -> RAG
raw_docs = load_pdfs(DATA_DIR)
CHUNK_STRATEGY_A = {"chunk_size": 300, "chunk_overlap": 50}
chunks = chunk_documents(raw_docs, **CHUNK_STRATEGY_A)

store = VectorStore()
store.build_index(chunks)
rag = RAG(store)

print(f"Documents: {len(raw_docs)} pages -> {len(chunks)} chunks (Strategy A)")
print("RAG retriever ready. Use rag.retrieve(query, top_k=3) to get (score, chunk) list.")

## Local LLM (Ollama) via LangChain

This notebook uses a local model through Ollama to keep experiments reproducible and offline-friendly.

Example setup (outside notebook):
- `ollama pull llama3.1` (or another model)
- Ensure Ollama is running (default: `http://localhost:11434`)

In [3]:
MODEL_NAME = "gemma3:4b"   # exactly as in `ollama list`
OLLAMA_BASE_URL = "http://localhost:11434"
TEMPERATURE_DEFAULT = 0.2

def build_llm(temperature: float) -> OllamaLLM:
    """
    Build a LangChain-compatible Ollama LLM.
    Using a factory function avoids duplication across experiments.
    """
    return OllamaLLM(
        model=MODEL_NAME,
        temperature=temperature,
        base_url=OLLAMA_BASE_URL,
        validate_model_on_init=True,  # fail fast if model is missing
    )

# Smoke test
build_llm(TEMPERATURE_DEFAULT).invoke("Reply: OK")


'OK\n'

## Retrieval: Week 2 Pipeline Inside Week 3

Контекст для LLM берём **здесь же**: для каждого запроса вызываем `rag.retrieve(query, top_k=3)` (тот же pipeline, что в Week 2). Файл Week 2 не нужен.


In [4]:
def get_retrieval_for_query(query: str, top_k: int = 3) -> dict:
    """Use Week 2 pipeline: rag.retrieve() returns (score, chunk). Build row for prompt."""
    retrieved = rag.retrieve(query, top_k=top_k)
    chunks = [c.page_content for _, c in retrieved]
    scores = [s for s, _ in retrieved]
    return {"query": query, "chunks": chunks, "scores": scores, "meta": {}}

# Example: one query to verify retrieval works
example = get_retrieval_for_query("What is RAG?")
print(f"Example retrieval: query={example['query'][:40]}...")
print(f"  Top-k chunks: {len(example['chunks'])}, scores: {[round(s, 4) for s in example['scores']]}")
print(f"  First chunk preview: {example['chunks'][0][:80]}...")


Loaded retrieval rows: 246
Keys: ['query', 'chunks', 'scores', 'meta']
Chunks in first row: 0


In [5]:
def format_context(chunks: list[str], scores: list[float] | None = None, max_chars: int = 7000) -> str:
    parts = []
    for i, ch in enumerate(chunks):
        score_str = f" (score={scores[i]:.4f})" if scores is not None else ""
        parts.append(f"[Chunk {i+1}{score_str}]\n{ch}")
    ctx = "\n\n".join(parts)
    return ctx[:max_chars]


## Prompt Templates

I define three task-specific prompt templates:

1) **Strict Q&A**  
   - Answer using ONLY context  
   - If missing: exact refusal message  
   - Require chunk citations

2) **Structured Summary**  
   - Extract key points, definitions, practical notes, and gaps

3) **Grounded Reasoning**  
   - Provide short reasoning steps tied to evidence  
   - Avoid outside knowledge


In [6]:
QA_STRICT_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a RAG assistant.\n"
        "You MUST answer using ONLY the provided CONTEXT.\n"
        "If the answer is not in the context, reply exactly:\n"
        "\"I don't know based on the provided context.\"\n\n"
        "CONTEXT:\n{context}\n\n"
        "QUESTION:\n{question}\n\n"
        "RULES:\n"
        "- No outside knowledge.\n"
        "- Be concise.\n"
        "- Cite chunks like [Chunk 2].\n\n"
        "ANSWER:\n"
    )
)

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["context", "topic"],
    template=(
        "Summarize the provided context about: {topic}\n\n"
        "CONTEXT:\n{context}\n\n"
        "OUTPUT FORMAT:\n"
        "- Key points (3–7 bullets)\n"
        "- Definitions (if any)\n"
        "- Practical notes (if any)\n"
        "- Missing information / open questions (if any)\n\n"
        "SUMMARY:\n"
    )
)

REASONING_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are an analyst. Use ONLY the provided context.\n"
        "Do not add outside knowledge.\n\n"
        "CONTEXT:\n{context}\n\n"
        "QUESTION:\n{question}\n\n"
        "OUTPUT FORMAT:\n"
        "1) Answer (1–3 sentences)\n"
        "2) Evidence: cite chunks like [Chunk 1]\n"
        "3) Reasoning: 3–6 short bullet steps tied to evidence\n\n"
        "RESPONSE:\n"
    )
)


In [7]:
# Retrieval is done by get_retrieval_for_query(question) — no find_retrieval_row needed.


In [8]:
def stable_id(*parts) -> str:
    s = "||".join(map(str, parts))
    return hashlib.sha256(s.encode("utf-8")).hexdigest()[:16]

def run_generation(
    question: str,
    retrieval_row: dict,
    prompt: PromptTemplate,
    template_name: str,
    temperature: float,
    topic: str = "RAG notes"
) -> dict:
    llm = build_llm(temperature)

    context = format_context(retrieval_row["chunks"], retrieval_row["scores"])
    kwargs = {"context": context}

    if "question" in prompt.input_variables:
        kwargs["question"] = question
    if "topic" in prompt.input_variables:
        kwargs["topic"] = topic

    prompt_text = prompt.format(**kwargs)

    t0 = time.time()
    answer = llm.invoke(prompt_text)
    latency = round(time.time() - t0, 3)

    answer = (answer or "").strip()

    return {
        "run_id": stable_id(question, template_name, MODEL_NAME, temperature),
        "question": question,
        "template": template_name,
        "model": MODEL_NAME,
        "temperature": temperature,
        "latency_s": latency,
        "prompt_chars": len(prompt_text),
        "context_chars": len(context),
        "n_chunks": len(retrieval_row["chunks"]),
        "retrieval_query_used": retrieval_row["query"],
        "answer": answer,
        "idk_flag": ("i don't know based on the provided context" in answer.lower()),
        "has_citation_flag": ("[chunk" in answer.lower()),  # lightweight heuristic
    }


## Test Set

A small, diverse set of questions is used to compare prompt behavior:
- definitions (low risk)
- “why” questions (higher hallucination risk)
- comparison questions
- pipeline-level questions

This is a controlled prompt experiment, not a full benchmark yet.


In [9]:
test_questions = [
    "What is Retrieval-Augmented Generation (RAG)?",
    "Why do we use chunk overlap in retrieval systems?",
    "Explain cosine similarity vs dot product similarity for embeddings.",
    "What is the role of FAISS in a RAG pipeline?",
    "When would you increase chunk size and why?",
]


In [10]:
templates = [
    ("qa_strict", QA_STRICT_PROMPT),
    ("summary", SUMMARY_PROMPT),
    ("reasoning", REASONING_PROMPT),
]

temperature_grid = [0.0, 0.2]  # compare deterministic vs slightly creative

records = []
for q in test_questions:
    r = get_retrieval_for_query(q)
    for (name, tpl) in templates:
        for temp in temperature_grid:
            rec = run_generation(
                question=q,
                retrieval_row=r,
                prompt=tpl,
                template_name=name,
                temperature=temp,
                topic="RAG / retrieval fundamentals"
            )
            records.append(rec)

df = pd.DataFrame(records)
df.head()


,run_id,question,template,model,temperature,latency_s,prompt_chars,context_chars,n_chunks,retrieval_query_used,answer,idk_flag,has_citation_flag
0,773148cfcd0d205f,What is Retrieval-Augmented Generation (RAG)?,qa_strict,gemma3:4b,0.0,1.294,325,0,0,Explain retrieval augmented generation in simp...,I don't know based on the provided context.,True,False
1,ed2bea5911a9d13d,What is Retrieval-Augmented Generation (RAG)?,qa_strict,gemma3:4b,0.2,0.496,325,0,0,Explain retrieval augmented generation in simp...,I don't know based on the provided context.,True,False
2,223dbe53cb99f7e7,What is Retrieval-Augmented Generation (RAG)?,summary,gemma3:4b,0.0,1.585,229,0,0,Explain retrieval augmented generation in simp...,Please provide the context about RAG/retrieval...,False,False
3,ec596e6f77b05533,What is Retrieval-Augmented Generation (RAG)?,summary,gemma3:4b,0.2,1.440,229,0,0,Explain retrieval augmented generation in simp...,Please provide the context about RAG/retrieval...,False,False
4,83b5662850e2bfeb,What is Retrieval-Augmented Generation (RAG)?,reasoning,gemma3:4b,0.0,7.530,296,0,0,Explain retrieval augmented generation in simp...,1) Retrieval-Augmented Generation (RAG) is a t...,False,True


In [28]:
df.to_csv(W3_EXPERIMENTS_CSV, index=False)

with open(W3_GENERATIONS_JSONL, "w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("Saved:", W3_EXPERIMENTS_CSV)
print("Saved:", W3_GENERATIONS_JSONL)


Saved: /Users/tkhamidulin/Desktop/First Project - RAG/artifacts/week3_prompt_experiments.csv
Saved: /Users/tkhamidulin/Desktop/First Project - RAG/artifacts/week3_generations.jsonl


In [29]:
df["answer_len"] = df["answer"].fillna("").str.len()

summary = (
    df.groupby(["template", "temperature"])
      .agg(
          n=("run_id", "count"),
          avg_latency_s=("latency_s", "mean"),
          avg_answer_len=("answer_len", "mean"),
          idk_rate=("idk_flag", "mean"),
          citation_rate=("has_citation_flag", "mean"),
      )
      .reset_index()
      .sort_values(["template", "temperature"])
)

summary


,template,temperature,n,avg_latency_s,avg_answer_len,idk_rate,citation_rate
0,qa_strict,0.0,5,0.6082,43.0,1.0,0.0
1,qa_strict,0.2,5,0.4452,43.0,1.0,0.0
2,reasoning,0.0,5,5.9418,1210.2,0.0,0.8
3,reasoning,0.2,5,5.5042,1152.6,0.0,0.8
4,summary,0.0,5,1.5362,267.0,0.0,0.0
5,summary,0.2,5,1.4084,267.0,0.0,0.0


In [30]:
q = test_questions[0]
df[df["question"] == q][["template", "temperature", "answer"]]


,template,temperature,answer
0,qa_strict,0.0,I don't know based on the provided context.
1,qa_strict,0.2,I don't know based on the provided context.
2,summary,0.0,Please provide the context about RAG/retrieval...
3,summary,0.2,Please provide the context about RAG/retrieval...
4,reasoning,0.0,1) Retrieval-Augmented Generation (RAG) is a t...
5,reasoning,0.2,1) Retrieval-Augmented Generation (RAG) is a t...


## Conclusions (Week 3)

### Prompt behavior findings
- **Strict Q&A**: most grounded and consistent with refusal rules.
- **Summary style**: best readability, but may drop details needed for precise Q&A.
- **Reasoning style**: better explanations, but more sensitive to weak context.

### Retrieval-to-generation dependency
Answer quality is directly limited by retrieval quality from Week 2: weak context leads to weaker grounded answers regardless of prompt style.

### Handoff to Week 4
Week 4 adds guardrails around this Week 3 pipeline: pre-checks before LLM calls and post-checks on outputs to reduce hallucinations, leakage, and unsafe responses.


In [31]:
summary_text = f"""# Week 3 — Prompt Engineering Summary

## Setup
- LLM: Ollama via LangChain
- Model: {MODEL_NAME}
- Templates: qa_strict, summary, reasoning
- Temperatures tested: {temperature_grid}

## Quick metrics (proxy)
{summary.to_string(index=False)}

## Notes (fill after manual review)
- qa_strict:
- summary:
- reasoning:

## Observed failure modes
- 

## Week 4 plan
- add guardrails for weak retrieval
- enforce minimum-evidence rules before answering
- improve refusal behavior and citation enforcement
"""

W3_SUMMARY_MD.write_text(summary_text, encoding="utf-8")
print("Saved:", W3_SUMMARY_MD)


Saved: /Users/tkhamidulin/Desktop/First Project - RAG/artifacts/week3_summary.md
